In [70]:
import re
import pandas as pd

In [71]:
df = pd.read_excel('../data/raw/registro_empresas.xlsx', index_col=None, sheet_name='empresas', 
                    converters={'Fecha de la resolución': str, 
                                'Fecha Gaceta': str,
                                'No. de Resolución': str})

df.rename(columns = {'Nombre limpio definitivo': 'Empresa'}, inplace = True)

df_empresas_list = df['Empresa'].unique().tolist()

len(df_empresas_list)

/home/fxr/.local/share/virtualenvs/registro_empresas_cuba-XkacQYve/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


1166

In [72]:
objetos = pd.read_excel('../data/raw/registro_empresas.xlsx', index_col=None, sheet_name='objetos')

objetos.rename(columns = {'Entidad (copiar exactamente como aparece en hoja "empresas"': 'Empresa',
                            'Objeto(s) en Anexo 1 (copiar cada uno en una fila)': 'Objetos'}, inplace = True)

objetos_empresa_list = objetos['Empresa'].unique().tolist()

len(objetos_empresa_list)

1260

In [73]:
def process_data(empresas_list):
    empresas_processed = []
    objetos_processed = []
    for i in range(len(empresas_list)):
        objetos_filtered = objetos.loc[objetos['Empresa'] == empresas_list[i]]
        objetos_list = objetos_filtered['Objetos'].values.tolist()
        objetos_flatten = ", ".join(objetos_list)
        objetos_processed.append(objetos_flatten)
        empresas_values = objetos_filtered['Empresa'].values.tolist()
        empresas_processed.append(empresas_values[0])
        df = pd.DataFrame(list(zip(empresas_processed, objetos_processed)),
                            columns =['Empresa', 'Objetos'])
    return df

result = process_data(objetos_empresa_list)

result

,Empresa,Objetos
0,JIANGSU GANGHONG ELECTRIC WIRE & POWER CABLE C...,"Capítulo 39 Plástico y sus manufacturas, Capít..."
1,SEVINCO S.L.,"Capítulo 74 Cobre y sus manufacturas, Capítulo..."
2,"CHINA NORTH VEHICLE CORPORATION, LTD. (NORINCO...",Capítulo 28 Productos químicos inorgánicos; co...
3,"EUROTRADE WORLD COMMERCE, S.L.",Capítulo 4 Leche y productos lácteos; huevos d...
4,"MIESA INGENIERÍA, S.A.","Capítulo 82 Herramientas y útiles, artículos d..."
...,...,...
1255,"SALINA COMPAÑÍA DE SERVICIOS INTEGRALES, S.L.",Capítulo 25 Sal; azufre; tierras y piedras; ye...
1256,"PROMOTORA ÓRBITA, S.L. UNIPERSONAL","Capítulo 2 Carne y despojos comestibles, Capít..."
1257,"ALDAKETA IMPORT­EXPORT, S.A.","Capítulo 2 Carne y despojos comestibles, Capít..."
1258,"CUGRANCA, S.A.","Capítulo 2 Carne y despojos comestibles, Capít..."


In [74]:
result['Objetos'] = result['Objetos'].str.findall(r'\d+')
result

,Empresa,Objetos
0,JIANGSU GANGHONG ELECTRIC WIRE & POWER CABLE C...,"[39, 73, 74, 76, 80, 82, 83, 84, 85]"
1,SEVINCO S.L.,"[74, 76, 78, 79, 80, 81, 82, 83, 84, 85, 87, 8..."
2,"CHINA NORTH VEHICLE CORPORATION, LTD. (NORINCO...","[28, 29, 39, 40, 72, 73, 74, 75, 84, 86, 87, 2..."
3,"EUROTRADE WORLD COMMERCE, S.L.","[4, 8, 12, 13, 15, 17, 18, 21, 25, 27, 28, 29,..."
4,"MIESA INGENIERÍA, S.A.","[82, 83, 84, 85, 90, 82, 83, 84, 85, 90]"
...,...,...
1255,"SALINA COMPAÑÍA DE SERVICIOS INTEGRALES, S.L.","[25, 28, 32, 39, 43, 44, 48, 49, 69, 70, 73, 8..."
1256,"PROMOTORA ÓRBITA, S.L. UNIPERSONAL","[2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 15, 16, ..."
1257,"ALDAKETA IMPORT­EXPORT, S.A.","[2, 3, 4, 7, 8, 9, 10, 11, 12, 15, 16, 17, 18,..."
1258,"CUGRANCA, S.A.","[2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, ..."


In [75]:
df_join = pd.merge(df, result, on='Empresa', how='outer')
df_join.head()

,ORDEN,Nombre de la empresa extranjera,Empresa,Nombre limpio viejo,Nombre del agente/representante cubano,,.1,Tipo,Objeto de la resolución,País casa matriz del agente,...,Fecha de la resolución,Firmante nombre,Firmante cargo,Código resolución,No. de Resolución,Fecha Gaceta,Código gaceta,Link sitio gaceta,Link Drive PI,Objetos
0,1.0,JIANGSU GANGHONG ELECTRIC WIRE & POWER CABLE C...,JIANGSU GANGHONG ELECTRIC WIRE & POWER CABLE C...,JIANGSU GANGHONG ELECTRIC WIRE & POWER CABLE C...,NaN,https://opencorporates.com/companies?utf8=%E2%...,NaN,Sucursal de compañía,Respuesta a solicitud de inscripción,NaN,...,2020-12-11 00:00:00,Rodrigo Malmierca Díaz,Ministro del MINCEX,GOC-2021-76-O6,398/2020,2021-01-18 00:00:00,GOC-2021-O-6,https://www.gacetaoficial.gob.cu/sites/default...,https://drive.google.com/file/d/1YS8-0jIo1b_En...,"[39, 73, 74, 76, 80, 82, 83, 84, 85]"
1,2.0,SEVINCO S.L.,"SEVINCO, S.L.",SEVINCO S.L.,NaN,https://opencorporates.com/companies?utf8=%E2%...,NaN,Sucursal de compañía,Respuesta a solicitud de inscripción,NaN,...,2020-12-11 00:00:00,Rodrigo Malmierca Díaz,Ministro del MINCEX,GOC-2021-77-O6,399/2020,2021-01-18 00:00:00,GOC-2021-O-6,https://www.gacetaoficial.gob.cu/sites/default...,https://drive.google.com/file/d/1YS8-0jIo1b_En...,NaN
2,3.0,AGENCIA MARÍTIMA MAPOR S.A.,"AGENCIA MARÍTIMA MAPOR, S.A.",AGENCIA MARÍTIMA MAPOR S.A.,NaN,https://opencorporates.com/companies?utf8=%E2%...,NaN,Sucursal de compañía,Respuesta a solicitud de inscripción,NaN,...,2020-12-11 00:00:00,Rodrigo Malmierca Díaz,Ministro del MINCEX,GOC-2021-78-O6,400/2020,2021-01-18 00:00:00,GOC-2021-O-6,https://www.gacetaoficial.gob.cu/sites/default...,https://drive.google.com/file/d/1YS8-0jIo1b_En...,NaN
3,623.0,AGENCIA MARÍTIMA MAPOR S.A.,"AGENCIA MARÍTIMA MAPOR, S.A.",AGENCIA MARÍTIMA MAPOR S.A.,NaN,NaN,NaN,Sucursal de compañía,Respuesta a solicitud de inscripción,NaN,...,2017-09-26 00:00:00,Rodrigo Malmierca Díaz,Ministro MINCEX,GOC-2018-75-O4,262/2017,2018-02-26 00:00:00,GOC-2018-O-4,https://www.gacetaoficial.gob.cu/sites/default...,https://drive.google.com/file/d/10Ad4Y924B-0Fg...,NaN
4,1725.0,AGENCIA MARÍTIMA MAPOR S.A.,"AGENCIA MARÍTIMA MAPOR, S.A.",AGENCIA MARÍTIMA MAPOR S.A.,NaN,NaN,NaN,Sucursal de compañía,Respuesta a solicitud de inscripción,NaN,...,2014-09-18 00:00:00,Rodrigo Malmierca Díaz,Ministro MINCEX,N.A.,249 de 2014,2014-10-24 00:00:00,GOC-2014-O-50,https://www.gacetaoficial.gob.cu/sites/default...,https://drive.google.com/file/d/1VUkhsv5k5OPmK...,NaN


In [76]:
result.to_csv('../data/processed/objetos_empresas.csv', index=False)
df_join.to_csv('../data/processed/registro_empresas.csv', index=False)